In [10]:
import sys

In [11]:
sys.exc_info()

(None, None, None)

In [1]:
import pandas as pd

In [2]:
path = 'artifact/08_19_2024_01_25_13/data_ingestion/feature_store/sensor.csv'

In [3]:
df = pd.read_csv(path)
df.head()

,class,aa_000,ab_000,ac_000,ad_000,ae_000,af_000,ag_000,ag_001,ag_002,...,ee_002,ee_003,ee_004,ee_005,ee_006,ee_007,ee_008,ee_009,ef_000,eg_000
0,pos,72504,NaN,1594.0,1052.0,0.0,0.0,0.0,244.0,178226.0,...,1432098.0,372252.0,527514.0,358274.0,332818.0,284178.0,3742.0,0.0,0.0,0.0
1,pos,791254,NaN,NaN,NaN,NaN,NaN,0.0,14868.0,921256.0,...,3625532.0,1380864.0,3262488.0,5727988.0,7563752.0,12114820.0,264714.0,198.0,NaN,NaN
2,pos,785270,NaN,NaN,8994.0,NaN,NaN,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
3,pos,281324,2.0,3762.0,2346.0,0.0,0.0,4808.0,215720.0,967572.0,...,624606.0,269976.0,638838.0,1358354.0,819918.0,262804.0,2824.0,0.0,0.0,0.0
4,pos,959094,2.0,288.0,266.0,0.0,0.0,0.0,0.0,0.0,...,790862.0,256590.0,566132.0,1192120.0,86112.0,16760.0,3794.0,0.0,0.0,0.0


In [4]:
df.shape

(36188, 171)

In [5]:
df.isnull().sum()

class         0
aa_000        0
ab_000    27896
ac_000     2141
ad_000     9200
          ...  
ee_007      379
ee_008      379
ee_009      379
ef_000     1730
eg_000     1729
Length: 171, dtype: int64

In [6]:
df.dtypes

class      object
aa_000      int64
ab_000    float64
ac_000    float64
ad_000    float64
           ...   
ee_007    float64
ee_008    float64
ee_009    float64
ef_000    float64
eg_000    float64
Length: 171, dtype: object

In [17]:
from scipy.stats import ks_2samp
import numpy as np

In [18]:
d1 = np.random.randn(100)
d2 = np.random.normal(loc=0.0, scale=1.0, size=100)
d3 = np.random.uniform(low=-1, high=2, size=100)

In [19]:
ks_2samp(d1, d2)

KstestResult(statistic=0.2, pvalue=0.03638428787491733, statistic_location=0.05668154816457801, statistic_sign=1)

In [20]:
ks_2samp(d1, d3)

KstestResult(statistic=0.24, pvalue=0.0061340334218580265, statistic_location=0.3986776616193534, statistic_sign=1)

In [21]:
ks_2samp(d2, d3)

KstestResult(statistic=0.16, pvalue=0.1548386665118475, statistic_location=1.250324822377371, statistic_sign=1)

In [22]:
train_path = r'artifact/08_17_2024_04_14_57/data_ingestion/ingested/train.csv'
test_path = r'artifact/08_17_2024_04_14_57/data_ingestion/ingested/test.csv'

In [23]:
train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)

In [24]:
train_df.columns

Index(['class', 'aa_000', 'ab_000', 'ac_000', 'ad_000', 'ae_000', 'af_000',
       'ag_000', 'ag_001', 'ag_002',
       ...
       'ee_002', 'ee_003', 'ee_004', 'ee_005', 'ee_006', 'ee_007', 'ee_008',
       'ee_009', 'ef_000', 'eg_000'],
      dtype='object', length=171)

In [25]:
test_df.columns

Index(['class', 'aa_000', 'ab_000', 'ac_000', 'ad_000', 'ae_000', 'af_000',
       'ag_000', 'ag_001', 'ag_002',
       ...
       'ee_002', 'ee_003', 'ee_004', 'ee_005', 'ee_006', 'ee_007', 'ee_008',
       'ee_009', 'ef_000', 'eg_000'],
      dtype='object', length=171)

In [26]:
train_df['aa_000'].isnull().sum()

0

In [27]:
test_df['aa_000'].isnull().sum()

0

In [28]:
d1 = train_df['ee_003']
d2 = test_df['ee_003']

In [29]:
ks_2samp(d1, d2)

KstestResult(statistic=nan, pvalue=nan, statistic_location=nan, statistic_sign=nan)

In [30]:

def get_drift_report(base_data: pd.DataFrame, current_data: pd.DataFrame, threshold: float=0.1):
    report = {}
    
    for col in base_data.columns:
        test = ks_2samp(base_data[col], current_data[col])
        
        # not have enough evidence to reject the null hypothesis (H0)
        # both the data seems to come from the same distribution
        if test.pvalue >= threshold:
            report[col] = {
                'p-value': test.pvalue,
                'drift status': False
            }
        else:
            report[col] = {
                'p-value': test.pvalue,
                'drift status': True
            }
    
    return report


get_drift_report(train_df, test_df)

{'class': {'p-value': 1.0, 'drift status': False},
 'aa_000': {'p-value': 0.7603742420966467, 'drift status': False},
 'ab_000': {'p-value': nan, 'drift status': True},
 'ac_000': {'p-value': nan, 'drift status': True},
 'ad_000': {'p-value': nan, 'drift status': True},
 'ae_000': {'p-value': nan, 'drift status': True},
 'af_000': {'p-value': nan, 'drift status': True},
 'ag_000': {'p-value': nan, 'drift status': True},
 'ag_001': {'p-value': nan, 'drift status': True},
 'ag_002': {'p-value': nan, 'drift status': True},
 'ag_003': {'p-value': nan, 'drift status': True},
 'ag_004': {'p-value': nan, 'drift status': True},
 'ag_005': {'p-value': nan, 'drift status': True},
 'ag_006': {'p-value': nan, 'drift status': True},
 'ag_007': {'p-value': nan, 'drift status': True},
 'ag_008': {'p-value': nan, 'drift status': True},
 'ag_009': {'p-value': nan, 'drift status': True},
 'ah_000': {'p-value': nan, 'drift status': True},
 'ai_000': {'p-value': nan, 'drift status': True},
 'aj_000': {'p-

In [31]:
import yaml
import os

In [32]:
def read_yaml_file(file_path: str) -> dict:
    try:
        with open(file_path, "rb") as yaml_file:
            return yaml.safe_load(yaml_file)
    except Exception as e:
        raise Exception(e) 

In [33]:
SCHEMA_FILE_PATH = os.path.join("config", "schema.yaml")
SCHEMA_FILE_PATH

'config\\schema.yaml'

In [34]:
schema_config = read_yaml_file(SCHEMA_FILE_PATH)
len(schema_config['columns'])

164

In [36]:
train_df.shape

(256, 171)

In [1]:
from datetime import datetime

In [3]:
round(datetime.now().timestamp())

1724679186